# Vision Pipeline: GPT-4o Engineering Drawing Inspector

**What this does:** Inspects engineering drawing PDFs against SolidWorks CAD data using GPT-4o vision.

**Pipeline:**
```
Drawing Image -> GPT-4o Vision Extract -> Normalize -> Validate -> Expand -> Match vs SW -> Score -> QC Report
```

**No GPU required** — runs on CPU + OpenAI API.

**How to use:**
1. Run Cell 1 to install the package
2. Run Cell 2 to upload your drawing image and SolidWorks JSON
3. Run all remaining cells
4. Review the QC report in Cell 13

**Required:**
- `OPENAI_API_KEY` (set in Colab Secrets or enter when prompted)
- Drawing image (PNG/JPG)
- SolidWorks JSON file (from SolidWorksExtractor)

**Optional:**
- `sw_mating_context.json` — assembly mating relationships
- `sw_mate_specs.json` — thread specs and interface constraints
- `sw_part_context_complete.json` — part number bridging database

In [ ]:
# Cell 1: Install ai_inspector package from GitHub
import subprocess, sys, os

# Clone the repo (force fresh to avoid stale cache)
REPO_URL = 'https://github.com/Continental-Direct/AI-tool.git'
REPO_DIR = '/content/AI-tool'

if os.path.exists(REPO_DIR):
    !rm -rf {REPO_DIR}

!git clone {REPO_URL} {REPO_DIR}

# Install the package + dependencies
!pip install -q openai Pillow python-dotenv
!pip install -q -e {REPO_DIR}

# Add to path
if REPO_DIR not in sys.path:
    sys.path.insert(0, REPO_DIR)

print('\nInstallation complete.')
print(f'Python: {sys.version}')

# Verify import
from ai_inspector.pipeline.vision_pipeline import VisionPipeline
from ai_inspector.report.qc_report import generate_from_pipeline
print('ai_inspector package imported successfully.')

In [ ]:
# Cell 2: Upload files and configure API key
import os
from pathlib import Path

OUTPUT_DIR = '/content/inspection_output'
os.makedirs(OUTPUT_DIR, exist_ok=True)

# ============================================================
# API KEY — tries Colab Secrets first, then prompts
# ============================================================
OPENAI_API_KEY = None
try:
    from google.colab import userdata
    OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
    print(f'OPENAI_API_KEY loaded from Colab Secrets (len={len(OPENAI_API_KEY)})')
except Exception:
    pass

if not OPENAI_API_KEY:
    import getpass
    OPENAI_API_KEY = getpass.getpass('Enter your OpenAI API key: ')
    print(f'OPENAI_API_KEY entered (len={len(OPENAI_API_KEY)})')

# ============================================================
# UPLOAD FILES
# ============================================================
from google.colab import files

print('\n--- Upload your DRAWING IMAGE (PNG/JPG) ---')
uploaded_images = files.upload()
SAMPLE_IMAGE = '/content/' + list(uploaded_images.keys())[0]
print(f'Drawing: {SAMPLE_IMAGE}')

print('\n--- Upload your SOLIDWORKS JSON ---')
print('(Click Cancel or upload an empty file to skip SW comparison)')
try:
    uploaded_sw = files.upload()
    if uploaded_sw:
        SW_JSON_PATH = '/content/' + list(uploaded_sw.keys())[0]
        print(f'SW JSON: {SW_JSON_PATH}')
    else:
        SW_JSON_PATH = ''
except Exception:
    SW_JSON_PATH = ''
    print('No SW JSON uploaded — running extraction only (no matching).')

# ============================================================
# OPTIONAL: Assembly context databases
# ============================================================
print('\n--- (Optional) Upload assembly context files ---')
print('Upload sw_mating_context.json, sw_mate_specs.json, sw_part_context_complete.json')
print('Or click Cancel to skip assembly context.')

MATING_CONTEXT_PATH = ''
MATE_SPECS_PATH = ''
PART_CONTEXT_PATH = ''

try:
    uploaded_ctx = files.upload()
    for fname in uploaded_ctx.keys():
        fpath = '/content/' + fname
        if 'mating_context' in fname:
            MATING_CONTEXT_PATH = fpath
        elif 'mate_specs' in fname:
            MATE_SPECS_PATH = fpath
        elif 'part_context' in fname:
            PART_CONTEXT_PATH = fpath
        print(f'  Loaded: {fname}')
except Exception:
    print('No assembly context files uploaded.')

# ============================================================
print(f'\n=== Configuration ===')
print(f'Drawing:        {SAMPLE_IMAGE}')
print(f'SW JSON:        {SW_JSON_PATH or "<none>"}')
print(f'Mating context: {MATING_CONTEXT_PATH or "<none>"}')
print(f'Mate specs:     {MATE_SPECS_PATH or "<none>"}')
print(f'Part context:   {PART_CONTEXT_PATH or "<none>"}')
print(f'Output dir:     {OUTPUT_DIR}')

In [ ]:
# Cell 3: Import and create pipeline
from ai_inspector.pipeline.vision_pipeline import VisionPipeline
from ai_inspector.pipeline.yolo_pipeline import PipelineResult

pipeline = VisionPipeline(api_key=OPENAI_API_KEY)
pipeline.load()

print(f'VisionPipeline ready: {pipeline.is_loaded}')
print('No GPU models needed — extraction via GPT-4o API')
print(f'PipelineResult fields: {list(PipelineResult.__dataclass_fields__.keys())}')

In [ ]:
# Cell 4: Run the vision pipeline
import time, os

print(f'Running GPT-4o vision pipeline on: {os.path.basename(SAMPLE_IMAGE)}')
print(f'Mating context: {"ON" if MATING_CONTEXT_PATH else "OFF"}')
print(f'Mate specs:     {"ON" if MATE_SPECS_PATH else "OFF"}')
print(f'Part context:   {"ON" if PART_CONTEXT_PATH else "OFF"}')
print()

start = time.time()

result = pipeline.run(
    image_path=SAMPLE_IMAGE,
    sw_json_path=SW_JSON_PATH or None,
    page_id='page_0',
    output_dir=OUTPUT_DIR,
    save_crops=False,
    use_vlm=False,
    mating_context_path=MATING_CONTEXT_PATH or None,
    mate_specs_path=MATE_SPECS_PATH or None,
    part_context_path=PART_CONTEXT_PATH or None,
)

elapsed = time.time() - start

extracted = result.packet_summary.get('extracted_callouts', 0)
print(f'Pipeline completed in {elapsed:.1f}s')
print(f'Extracted callouts: {extracted}  (via GPT-4o vision)')
print(f'Match results: {len(result.match_results)}')
print(f'Mating context: {"found" if result.mating_context else "not found"}')
print(f'Mate specs: {"found" if result.mate_specs else "not found"}')

In [ ]:
# Cell 5: Scores and summary
import json

print('=== SCORES ===')
if result.scores:
    for key, val in result.scores.items():
        print(f'  {key:25s}: {val}')
else:
    print('  (no scores — run without SW data)')

print('\n=== EXPANSION SUMMARY ===')
print(json.dumps(result.expansion_summary, indent=2))

print('\n=== VALIDATION STATS ===')
print(json.dumps(result.validation_stats, indent=2))

print('\n=== PACKET SUMMARY ===')
print(json.dumps(result.packet_summary, indent=2))

In [ ]:
# Cell 6: Match results table
from ai_inspector.comparison.matcher import MatchStatus

if not result.match_results:
    print('No match results (no SW data provided or no features detected).')
else:
    print(f'{"#":>3s} {"Status":15s} {"Type":18s} {"Delta":>10s} {"Notes"}')
    print('=' * 90)

    for i, r in enumerate(result.match_results):
        callout_type = ''
        if r.drawing_callout:
            callout_type = r.drawing_callout.get('calloutType', '')
        elif r.sw_feature:
            callout_type = r.sw_feature.feature_type

        delta_str = f'{r.delta:+.4f}' if r.delta is not None else 'N/A'

        marker = {
            MatchStatus.MATCHED: '[OK]',
            MatchStatus.MISSING: '[MISS]',
            MatchStatus.EXTRA: '[EXTRA]',
            MatchStatus.TOLERANCE_FAIL: '[TOL]',
            MatchStatus.SKIPPED: '[SKIP]',
        }.get(r.status, '[?]')

        print(f'{i:3d} {marker + " " + r.status.value:15s} '
              f'{callout_type:18s} {delta_str:>10s} {r.notes[:50]}')

In [ ]:
# Cell 7: Assembly Mating Context

mc = result.mating_context
if not mc:
    print('No mating context (path not provided or part not found in database).')
else:
    print('=== ASSEMBLY MATING CONTEXT ===')
    print()
    print(f'  Part number:   {mc.get("part_number", "N/A")}')
    print(f'  Description:   {mc.get("description", "N/A")}')
    print(f'  Type:          {mc.get("type", "N/A")}')
    print(f'  Assembly:      {mc.get("assembly", "N/A")}')

    siblings = mc.get('siblings', [])
    if siblings:
        print(f'\n  --- Sibling Components ({len(siblings)}) ---')
        for s in siblings:
            print(f'    {s.get("pn", "?")} - {s.get("desc", "?")} ({s.get("type", "?")})')

In [ ]:
# Cell 8: Mate Specifications

ms = result.mate_specs
if not ms:
    print('No mate specs (path not provided or part/siblings not found).')
else:
    source = ms.get('source', 'direct')
    print(f'=== MATE SPECIFICATIONS (source: {source}) ===')
    print()

    if source == 'sibling_cross_reference':
        print(f'  Part {ms.get("part_number")} not directly in mate_specs.')
        print(f'  Showing specs from {len(ms.get("sibling_specs", []))} sibling(s):')
        for spec in ms.get('sibling_specs', []):
            pn = spec.get('part_number', '?')
            desc = spec.get('description', '')
            print(f'\n  --- Sibling: {pn} ({desc}) ---')
            for m in spec.get('mates_with', []):
                line = f'    {m.get("mate_type", "?")} with {m.get("part", "?")}'
                if m.get('description'):
                    line += f' ({m["description"]})'
                if m.get('thread'):
                    line += f'  ** THREAD: {m["thread"]} pitch={m.get("pitch","")} len={m.get("length","")} **'
                print(line)
    else:
        print(f'  Part: {ms.get("part_number", "?")}')
        print(f'  Description: {ms.get("description", "")}')
        for m in ms.get('mates_with', []):
            line = f'    {m.get("mate_type", "?")} with {m.get("part", "?")}'
            if m.get('description'):
                line += f' ({m["description"]})'
            if m.get('thread'):
                line += f'  ** THREAD: {m["thread"]} pitch={m.get("pitch","")} len={m.get("length","")} **'
            print(line)

In [ ]:
# Cell 9: GPT-4o Extracted Callouts
import json
from pathlib import Path

callouts_path = Path(OUTPUT_DIR) / 'callouts_extracted.json'
if callouts_path.exists():
    with open(callouts_path, 'r', encoding='utf-8') as f:
        callouts = json.load(f)
    print(f'=== GPT-4o Extracted Callouts ({len(callouts)}) ===')
    print()
    for i, c in enumerate(callouts):
        ct = c.get('calloutType', '?')
        raw = c.get('raw', '')[:60]
        qty = c.get('quantity', 1)
        parts = [f'{ct}']
        if c.get('diameter') is not None:
            parts.append(f'dia={c["diameter"]}')
        if c.get('radius') is not None:
            parts.append(f'R={c["radius"]}')
        if c.get('nominal') is not None:
            parts.append(f'nom={c["nominal"]}')
        if c.get('thread'):
            t = c['thread']
            parts.append(f'thread={t.get("raw", "")}')
        if qty > 1:
            parts.append(f'qty={qty}')
        print(f'  {i+1:2d}. {" | ".join(parts)}')
        print(f'      raw: "{raw}"')
else:
    print('No callouts_extracted.json found.')

In [ ]:
# Cell 10: List saved artifacts
from pathlib import Path

out = Path(OUTPUT_DIR)
print(f'Artifacts saved to: {OUTPUT_DIR}/')
print()

for f in sorted(out.rglob('*')):
    if f.is_file():
        size_kb = f.stat().st_size / 1024
        print(f'  {f.relative_to(out)}  ({size_kb:.1f} KB)')

In [ ]:
# Cell 11: GPT-4o QC Report — full JSON context
import json, os
from pathlib import Path
from ai_inspector.report.qc_report import generate_from_pipeline

# --- Load extracted callouts from debug output ---
extracted_callouts = []
callouts_path = Path(OUTPUT_DIR) / 'callouts_extracted.json'
if callouts_path.exists():
    with open(callouts_path, 'r', encoding='utf-8') as f:
        extracted_callouts = json.load(f)

# --- Load validated callouts from debug output ---
validated_callouts = []
validated_path = Path(OUTPUT_DIR) / 'validated_callouts.json'
if validated_path.exists():
    with open(validated_path, 'r', encoding='utf-8') as f:
        validated_callouts = json.load(f)

# --- Load SW identity (part number, description, material, etc.) ---
sw_identity = {}
if SW_JSON_PATH and os.path.exists(SW_JSON_PATH):
    with open(SW_JSON_PATH, 'r', encoding='utf-8-sig') as f:
        sw_raw = json.load(f)
    sw_identity = sw_raw.get('identity', {})

# --- Generate report with full JSON context ---
print(f'Part: {sw_identity.get("partNumber", "?")} -- {sw_identity.get("description", "?")}')
print(f'Extracted callouts: {len(extracted_callouts)}')
print(f'Validated callouts: {len(validated_callouts)}')
print(f'Match results: {len(result.match_results)}')
print()
print('Sending full inspection context as JSON to GPT-4o...')

report, inspection_context = generate_from_pipeline(
    result=result,
    extracted_callouts=extracted_callouts,
    validated_callouts=validated_callouts,
    sw_identity=sw_identity,
    api_key=OPENAI_API_KEY,
    model='gpt-4o',
)

print(f'Report generated -- Status: {report.status}')
print(f'Model: {report.model_used}')
print()
print('=' * 70)
print(report.report_text)
print('=' * 70)

# Save report
report_path = Path(OUTPUT_DIR) / 'qc_report.md'
with open(report_path, 'w', encoding='utf-8') as f:
    f.write(report.report_text)

# Save full context that was sent to GPT
context_path = Path(OUTPUT_DIR) / 'inspection_context.json'
with open(context_path, 'w', encoding='utf-8') as f:
    json.dump(inspection_context, f, indent=2, ensure_ascii=False, default=str)

# Save structured report data
report_data_path = Path(OUTPUT_DIR) / 'qc_report.json'
with open(report_data_path, 'w', encoding='utf-8') as f:
    json.dump(report.to_dict(), f, indent=2, ensure_ascii=False)

print(f'\nReport saved to: {report_path}')
print(f'Full context saved to: {context_path}')
print(f'Report data saved to: {report_data_path}')

In [ ]:
# Cell 12: Download results
from google.colab import files as colab_files
from pathlib import Path
import shutil

# Zip all output artifacts
zip_path = shutil.make_archive('/content/inspection_results', 'zip', OUTPUT_DIR)
print(f'Results zipped: {zip_path}')
print()

# List contents
out = Path(OUTPUT_DIR)
for f in sorted(out.rglob('*')):
    if f.is_file():
        size_kb = f.stat().st_size / 1024
        print(f'  {f.relative_to(out)}  ({size_kb:.1f} KB)')

print('\nDownloading results zip...')
colab_files.download(zip_path)

In [ ]:
# Cell 13: Cleanup
import gc

pipeline.unload()
gc.collect()
print('Pipeline unloaded. Done.')